# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Основная цель

Построить модель для определения стоимости автомобиля с высоким качеством предсказания и оптимальным временем обучения и скорости предсказания.

# План выполнения

1. Загрузить, изучить, а так же подготовить данные для обучения моделей. 
2. Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг, с различными гиперпараметрами.
3. Проанализировать время обучения, время предсказания и качество моделей.
4. Выбрать лучшую модель и проверить ее на тестовой выборке.

# Описание даных

Признаки:

* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

Целевой признак: 

* Price — цена (евро)

## Подготовка данных

### Загрузка данных и библиотек

In [5]:
# Установим библиотеку формирующую сводную информацию о данных в датасете
!pip install -q pandas-profiling[notebook]
#  И библиотеку столбцопорядконаводителя
!pip install -q skimpy

In [4]:
!pip install -q scikit-learn==1.1.3

In [7]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from pandas-profiling import ProfileReport

from skimpy import clean_columns


from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [8]:
# Сохраним датасет в переменную
df = pd.read_csv('/datasets/autos.csv')

Исходя из поставленной задачи, можно выделить несколько столбцов, которые никак не помогут определить нам его стоимость и удалить их.

* DateCrawled — дата скачивания анкеты из базы (Это техническая информация, не влияет на цену)
* RegistrationMonth — месяц регистрации автомобиля (Год важен, месяц вряд ли имеет сколько-то важное значение)
* DateCreated — дата создания анкеты (это информация о человеке, как о пользователе, даже не как о владельце авто)
* NumberOfPictures — количество фотографий автомобиля (качественная презентация может оказывать влияние в частном случае, но не является параметром влияющим на рыночную стоимость автомобиля)
* PostalCode — почтовый индекс владельца анкеты (Техническая информация о пользователе, нам не нужна)
* LastSeen — дата последней активности пользователя (Техническая информация о пользователе, нам не нужна)

### Корректировка актуальности данных и внешнего вида

In [9]:
# Избавимся от неинформативных столбцов
df = df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis= 1)

In [10]:
# Так же приведем заголовки к привычному виду
df = clean_columns(df)

10 column names have been cleaned

### Изучение и подготовка данных

In [7]:
# Изучим данные
ProfileReport(df, title="Анатлитический отчет")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Начнем ознакомление с целевого признака. Похоже на площадке "Не бит, Не крашен", допускается публикация объявлений без цены. 3% строк имеют значение 0. Так же здравый рассудок подсказывает, что если цена автомобиля слишком низкая, значит цена либо указана некорректно (например продавец решил так привлечь внимание к объявлению или просто ошибся) либо возможно автомобиль не на ходу. Я допускаю, что очень старый автомобиль с множественными недостатками может стоить 300 евро, дешевле вряд ли. Соответственно все что дешевле этого порога нам не пригодится.

In [8]:
# Исключим из выборки все предложения дешевле 300 евро.
df = df.query('price > 300')

Пропуски:
* Тип Кузова
* Коробка передач
* Модель
* Восстановленная
* Тип топлива

Нулевые значения:
* Мощность двигателя

Сомнительные значения:
* Год выпуска
* Мощность двигателя

In [9]:
# В столбцах Тип кузова, Модель и Тип топлива уже есть категория для неопределившихся, имя ей 'other', туда и определим пропуски
df[['vehicle_type', 'model', 'fuel_type']] = df[['vehicle_type', 'model', 'fuel_type']].fillna('other')
# Тип коробки передач, признак, от которого сильно зависит цена, взять эти данные нам негде, будем резать.
df.dropna(subset=['gearbox'], inplace=True)
# Если в объявлении явно не указано, что автомобиль восстановленный, значит это не так, либо это пытаются скрыть,
# но тогда и продают по цене целого, поэтому заполним пропуски значением No
df['repaired'] = df['repaired'].fillna('no')
# Мощность двигателя напрямую влияет на цену, нулевые значения нас не интересуют, заодно и неадекватно высокие чикнем
# За верхний предел возьмем значения самого мощного серийного автомобиля по версии википедии
# за нижний cоответственно самого немощного
df = df.query('power < 600 & power > 4')
# В столбце год выпуска за адекватный интервал будем считать с 1960 по 2022
df = df.query('registration_year < 2017 & registration_year > 1959')
# Исключим дубликаты
df = df.drop_duplicates()
# Сброс индексов
df.reset_index(drop=True)

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
0,18300,coupe,2011,manual,190,other,125000,gasoline,audi,yes
1,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,no
2,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
3,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
4,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes
...,...,...,...,...,...,...,...,...,...,...
237725,5250,other,2016,auto,150,159,150000,other,alfa_romeo,no
237726,3200,sedan,2004,manual,225,leon,150000,petrol,seat,yes
237727,1199,convertible,2000,auto,101,fortwo,125000,petrol,smart,no
237728,9200,bus,1996,manual,102,transporter,150000,gasoline,volkswagen,no


In [11]:
# Обозначим целевой признак
features = df.drop(['price'], axis=1)
target = df['price']

# Поделим выборку на обучающую и тестовую в соотношении 75:25
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                              test_size=0.25, random_state=12345)

In [12]:
# Категориальные признаки
ohe_features_train = features_train.select_dtypes(include='object').columns.to_list()
# Численные признаки
num_features = features_train.select_dtypes(exclude='object').columns.to_list()

In [13]:
train = features_train.copy()
test = features_test.copy()

In [14]:
# Обучаем энкодер
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
encoder_ohe.fit(train[ohe_features_train])

# Добавляем закодированные признаки в переменную
train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(train[ohe_features_train])

# Удаляем незакодированные категориальные признаки
train = train.drop(ohe_features_train, axis=1)

# Обучаем скалер
scaler = StandardScaler()
train[num_features] = scaler.fit_transform(train[num_features])

In [15]:
# Кодируем тестовую выборку
test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(test[ohe_features_train])

test = test.drop(ohe_features_train, axis=1)

# Масштабируем тестовую выборку
test[num_features] = scaler.transform(test[num_features])

In [16]:
# Проверка на равное количество признаков в выборках
train.shape, test.shape

((178297, 304), (59433, 304))

* Выполнена загрузка данных и импорт необходимых для работы библиотек.
* Из датасета исключены бесполезные для дальнейшей работы значения.
* Внешний вид столбцов приведен в эстетичный вид.
* Обработаны пропуски, выбивающиеся и аномальные значения, дубликаты.
* Выполнено преобразование категориальных признаков.
* Обозначен целевой признак, датасет поделен на выборки, произведено масштабирование признаков.

Данные подготовлены к дальнейшей работе.

## Обучение моделей

### Дерево решений

In [17]:
# Выделим в отдельный блок изменяемые значения
RND = 12345
# После подбора значений в сетке остались только лучшие, для уменьшения времени загрузки проекта
grid_space = {'max_depth':[15]}

# Найдем оптимальные значения гиперпараметров и обучим модель
model = DecisionTreeRegressor(random_state = RND)
decision_tree = GridSearchCV(model, param_grid = grid_space, scoring='neg_mean_squared_error', cv=3)
decision_tree_v = decision_tree.fit(train, target_train)

# Выведим лучшее значение метрики и при каких гиперпараметрах оно было получено
print('Оптимальное значение гиперпараметров:', decision_tree_v.best_params_)
print('Лучший показатель RMSE: {:.2f}'.format((-decision_tree_v.best_score_) ** 0.5))

# Посмотрим сколько времени это все занимает
decision_tree_time = pd.DataFrame(decision_tree_v.cv_results_).iloc [:, 0:5]
display(decision_tree_time)

Оптимальное значение гиперпараметров: {'max_depth': 15}
Лучший показатель RMSE: 1953.38


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth
0,2.73477,0.154946,0.063423,0.00433,15


### LightGBM

In [18]:
# После подбора значений в сетке остались только лучшие, для уменьшения времени загрузки проекта
grid_space = {
    'max_depth': [3],
    'learning_rate': [0.15],
    'cat_smooth': [1]
}

# Найдем оптимальные значения гиперпараметров и обучим модель
model = LGBMRegressor(random_state = RND)
lgbm_reg = GridSearchCV(model, param_grid = grid_space, scoring='neg_mean_squared_error', cv=3)
lgbm_reg_v = lgbm_reg.fit(train, target_train)

# Выведим лучшее значение метрики и при каких гиперпараметрах оно было получено
print('Оптимальное значение гиперпараметров:', lgbm_reg_v.best_params_)
print('Лучший показатель RMSE: {:.2f}'.format((-lgbm_reg_v.best_score_) ** 0.5))

# Посмотрим сколько времени это все занимает
lgbm_reg_time = pd.DataFrame(lgbm_reg_v.cv_results_).iloc [:, 0:7]
display(lgbm_reg_time)

Оптимальное значение гиперпараметров: {'cat_smooth': 1, 'learning_rate': 0.15, 'max_depth': 3}
Лучший показатель RMSE: 1872.93


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_cat_smooth,param_learning_rate,param_max_depth
0,77.528811,1.198903,0.503303,0.00366,1,0.15,3


### CatBoostRegressor 

In [19]:
# После подбора значений в сетке остались только лучшие, для уменьшения времени загрузки проекта
grid_space = {
            'n_estimators' : [150],
            'verbose' : [0],
            'max_depth' : [13]
}

# Найдем оптимальные значения гиперпараметров и обучим модель
model = CatBoostRegressor(random_state = RND)
cat_boost = GridSearchCV(model, param_grid = grid_space, scoring='neg_mean_squared_error', cv=3)
cat_boost_v = cat_boost.fit(train, target_train)

# Выведим лучшее значение метрики и при каких гиперпараметрах оно было получено
print('Оптимальное значение гиперпараметров:', cat_boost_v.best_params_)
print('Лучший показатель RMSE: {:.2f}'.format((-cat_boost_v.best_score_) ** 0.5))

# Посмотрим сколько времени это все занимает
cat_boost_time = pd.DataFrame(cat_boost_v.cv_results_).iloc [:, 0:6]
display(cat_boost_time)

Оптимальное значение гиперпараметров: {'max_depth': 13, 'n_estimators': 150, 'verbose': 0}
Лучший показатель RMSE: 1624.14


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators
0,19.234479,0.304614,0.103481,0.003893,13,150


## Анализ моделей

Для того, чтобы проанализировать модели, вернемся к критериям, которые важны заказчику.

* качество предсказания
* время обучения модели
* время предсказания модели

Соответственно качество модели по метрике RMSE должно быть не выше 2500, при прочих равных, чем меньше время обучения и предсказания модели, тем лучше. 

Что касается метрики RMSE, то все представленные модели выдали эту метрику в рамках допуска.

* Дерево решений - 1953
* LightGBM - 1872
* CatBoostRegressor - 1624

Относительно качества предсказания, значительно эффективнее показал себя CatBoostRegressor, Дерево решений и LightGBM оказались менее эффективны.

Однако исходя из таблиц представленных в разделе обучения моделей, можно увидеть, что самым быстрым является Дерево решений и при этом качество предсказания вполне удовлетворяет требованиям обозначенным в условиях. LightGBM работает гораздо медленнее и при этом прирост качества незначительный. CatBoostRegressor справляется медленнее Дерева решений, однако и качество предсказания значительно выше.

Запустим самую эффективную модель с тестовой выборкой.

In [20]:
%%time

prediction = cat_boost.predict(test)
metric_test = mean_squared_error(target_test, prediction, squared=False)
metric_test

CPU times: user 129 ms, sys: 0 ns, total: 129 ms
Wall time: 131 ms


1596.3259824366976

На мой взгляд хороший результат. 

# Вывод

В ходе выполнения проекта исходный датасет был загружен и изучен, проведена предобработка загруженных данных, удалены дубликаты и аномальные значения, исключены неинформативные значения, пропуски заполнены. 

При обучении моделей были подобраны оптимальные гиперпараметры, так же зафиксированы значения метрики RMSE и скорость выполнения обучения и предсказаний.

Полученные данные проанализированы. Так как все модели соответствуют допустимому значению качества предсказания, любую из моделей можно использовать для поставленной задачи в зависимости от приоритетов:

По итогу проекта, можно выделить модель CatBoostRegressor, которая выдает значение RMSE 1596 на тестовой выборке при следующих значениях гиперпараметров:
* n_estimators : [150],
* verbose : [0],
* max_depth : [13]